## Building the Object Table(s)

This notebook describes how the 'Object' tables are constructed, starting with the **Stripe 82** dataset and followed the **XMM-LSS** dataset. The two 'Object' tables are merged into one in the end.

### **1. Stripe 82**
```bash
-> s82ObjectTable.parquet
```
The __Stripe 82__ region in this data challenge is larger than the SDSS Stripe 82 area, where the **DES** and **HSC** Stripe 82 coverage have been included. The exact footprint is provided below:

| RA         | Dec           |
| ---------- | ------------- |
| (-60, -43] | (-1.25, 1.25) |
| (-43, 0]   | (-2, 2)       |
| (0, 45]    | (-7, 5)       |
| (45, 60)   | (-1.25, 1.25) |


The Stripe 82 dataset consists of quasars/AGNs, galaxies and stars, where not all sources have been spectroscopically classified. Next, we will walk through each catagory of measurements included in the object(s) table and provide details on how they are derived. The nomenclature used here follows (as much as possible) that used in the LSST Data Products document - [LSE-163](https://github.com/lsst/LSE-163). __The units for the measurements included have also been converted to that used in the [LSE-163](https://github.com/lsst/LSE-163) document__.

### 1.1 Astrometry
- `ra`, `dec`
- `psPm_ra`, `psPm_dec`
- `psParallax`

1. `ra` and `dec` are in __degrees__ (J2000)
2. `psPm_ra` and `psPm_dec` are proper motion measurements obtained by mathcing the objects in our sample to GAIA EDR3 and the NOIRLab Source Catalog (NSC) [DR2](https://datalab.noao.edu/nscdr2/index.php). Proper motion measurements from GAIA take the precedence when matches are found in both GAIA and NSC. 
3. `psParallax` measurements come from GAIA EDR3.

### 1.2 Photometry
- `psMag`, `psMagErr`
- `bdMag`, `bdMagErr`

    >__Note__ that magnitude is NOT included in the data release catalog as per the current version of the data model, however we are providing the information here for completeness. 

    We match the objects in our dataset to both SDSS and DES DR2 for photometry and pick one by the following order: **_DES_ -> _SDSS Stripe 82 Coadd_ -> _SDSS DR16_** (with decreasing precedence). 

    1. The point-source magnitude ('psMag') is filled by 'psfmag' from SDSS or 'wavg_mag_psf' (weighted-average single epoch PSF magnitude) from DES. 
    
    2. The extended-source magnitude ('bdMag') is filled using 'modelmag' from SDSS or 'magauto' (measured on coadded images) from DES.

- `psFlux`, `psFluxErr`
- `bdFlux`, `bdFluxErr`

    > __Note__: The LSST Data Products document listed two flux measurments for extended objects, `bdFluxB` and `bdFluxD`, from a bulge+disk fitting. In this data challenge, we only include one flux measurement for extended sources.

__A. SDSS asinh magnitude to flux:__
1. The convertion from asinh magnitude (SDSS) to flux (nJy) follows the procedures provided by the SDSS online documentation at [here](https://www.sdss.org/dr16/algorithms/magnitudes/#asinh) and [here](https://www.sdss.org/dr16/algorithms/fluxcal/#SDSStoAB).
2. The convertion from asinh magnitude error to flux error follows the formula provided by [NED](https://ned.ipac.caltech.edu/help/sdss/dr6/photometry.html), which can be derived by taking the derivative of the convertion from asinh magnitude to flux:

$$\delta f= |\frac{-f*\delta m/a}{tanh((-m/a)-ln(b))}|$$

where f is the flux, m is SDSS asinh magnitude, a = $2.5/ln(10)$, b is the 'softening' parameter and $\delta f$ and $\delta m$ are the associated errors.
    
__B. AB magnitude to flux:__
1. From AB magnitude to flux follows the definition of AB magnitude:

$$m_{AB} = -2.5*log_{10}(\frac{f}{3631 Jy})$$

2. The flux error can be obtained using:

$$\delta f = f*|(e^{\delta m/a} -1|$$

where a = $2.5/ln(10)$, derived from $\delta m \approx 2.5/ln\,(10)*ln\,(1+\delta f/f)$

### 1.3 Extendedness (morphology)
- `extendedness`

    Per the LSST Data Products document, 'A measure of extendedness, computed using a combination of available moments, or from a likelihood ratio of point/B+D source models (exact algorithm TBD). `extendedness` = 1 implies a high degree of confidence that the source is extended. `extendedness` = 0 implies a high degree of confidence that the source is point-like.'

    We use the `probPSF` from SDSS and `class_star` from DES (depending on whose photometry is used) to fill this columns. However, both `probPSF` and `class_star` (from Sextractor) use 1 to indicate point-like and 0 for extended, which is opposite to the definition of the `extendedness` parameters, thus we inverted `probPSF`/`class_star`:

    $$ extendedness = 1 - probPSF$$ 

    or 

    $$ extendedness = 1 - class\_star$$ 

### 1.4 Standard Color and Error
- `stdColor`

    The standard colors, u-g, g-r, r-i, i-z, z-y, are derived by first computing the ratios of the fluxes from different bands and take the log of the ratios:

    $$C = -2.5*log_{10}(flux\_ratio)$$

    where C is the color.

- `stdColorErr`

    The error on the color is derived through standard uncertainty propagation.

    -> __*Variance on the flux ratio (assuming no correlation between flux errors):*__

    $$\mathrm{Var}(f_{1}/f_{2}) = (f_{1}/f_{2})^{2}[(\delta f_{1}/f_{1})^{2} + (\delta f_{2}/f_{2})^{2}]$$

    -> __*Propagating the error on flux ratio to colors (in magnitude):*__

    $$\mathrm{Var(C)} = a^{2}*\frac{\mathrm{Var}(f_{1}/f_{2})}{(f_{1}/f_{2})^{2}}$$ 

    where C stands for the color, a = $2.5/ln(10)$ and $f_{1} and f_{2}$ are the fluxes of the sources in the two bands used to compute the color.

### 1.5 lcPeriodic & lcNonPeriodic
- `lcPeriodic`
- `lcNonPeriodic`

See the [04_LC_features.ipynb](04_LC_features.ipynb) notebooks for more details on the light curve features computed.

### 1.6 Photometric Redshift
- `photoZ_pest`
We are filling in this column with SDSS DR16 galaxy photometric redshift (photo-z), see [here](https://www.sdss.org/dr16/algorithms/photo-z/) for more on the exact algorithm used to compute the photo-z. A matching raidus of 1 arcsec is used. 

### __2. XMM-LSS__
```bash
-> xmmlssObjectTable.parquet
```
The footprint is:

| RA             | Dec            |
| -------------- | -------------- |
| [34.2, 37.125] | [-5.72, -3.87] |

The __XMM-LSS__ region defined above is actually part of our extended Stripe 82 area, however, because of the extensive and deep multiwavelength (e.g., X-ray, IR, radio) coverage on this region, we built a seperate table for this region and combine it with the Stripe 82 dataset in the end. 

The basically procedure for building the __XMM-LSS__ dataset is essentially the same as that used for the Stripe 82 dataset. Thus we will briefly go through the different catategory of columns and only emphasize on showing the differences. 

### 2.1 Astrometry
See Section 1.1.

### 2.2 Photometry

- `psMag`, `psMagErr`
- `bdMag`, `bdMagErr`
- `psFlux`, `psFluxErr`
- `bdFlux`, `bdFluxErr`


__-> u band:__

The __XMM-LSS__ field is coverd by the Canada-France-Hawaii Telescope Legacy Survey ([CFHTLS](https://www.cfht.hawaii.edu/Science/CFHTLS/)). We mathed our initial sample to the CFHTLS DR7 hosted on [VizieR](http://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=II/317/cfhtls_w). We used the 3" aperture magnitude to fill the 'psMag' column and the 'mag_auto' (Kron-like aperture) to fill the 'bdMag' column. The fluxes are computed following the standard definition of AB magnitude (see Section 1.2). 

__-> 'grizy' bands:__

All quantities are computed using the LSST pipeline run on HSC images. We used the 'm_base_psfflux_' to populate the 'psMag' and 'psFlux' columns and the 'm_base_gaussianflux_' to fill the 'bdMag' and 'bdFlux' columns.

### 2.3 Extendedness (morphology)
- `extendedness`

__-> u band:__

Filled by the 'class_star' measurement from the CFHTLS catalog, inverted following the instruction given in Section 1.3 before populating.

__-> 'grizy' bands:__

LSST pipeline output this parameter for every photometric measurement. 

### 2.4 Standard Color and Error
See Section 1.4.

### 2.5 lcPeriodic & lcNonPeriodic
Not public time-domain data found in this area, which can match the depths of HSC, thus N/A.

### 2.6 Photo-z
- For AGNs in this region, the photo-z are provided by the XMM-SERVS team as documented in the [paper](https://academic.oup.com/mnras/article-abstract/478/2/2132/4985846?redirectedFrom=fulltext). 
- For other source, we matched them to the SDSS DR16 photo-z catalog (see Section 1.6).

### 3. The 'Object' Table
```bash
-> ObjectTable.parquet
```
The last step is to merge the __Stripe 82__ object table and the __XMM-LSS__ object table. We did that by matching the __XMM-LSS__ source with the __Stripe 82__ sources using a 1 arcsec raidus. The Stripe 82 sources that have counterparts in the XMM-LSS sample are removed from the final merged table. 